## Week 4, Lab 2: Predicting Chronic Kidney Disease in Patients
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus on steps exploring data, building models and evaluating the models we build.

There are three links you may find important:
- [A set of chronic kidney disease (CKD) data and other biological factors](./chronic_kidney_disease_full.csv).
- [The CKD data dictionary](./chronic_kidney_disease_header.txt).
- [An article comparing the use of k-nearest neighbors and support vector machines on predicting CKD](./chronic_kidney_disease.pdf).

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Step 1: Define the problem.

Suppose you're working for Mayo Clinic, widely recognized to be the top hospital in the United States. In your work, you've overheard nurses and doctors discuss test results, then arrive at a conclusion as to whether or not someone has developed a particular disease or condition. For example, you might overhear something like:

> **Nurse**: Male 57 year-old patient presents with severe chest pain. FDP _(short for fibrin degradation product)_ was elevated at 13. We did an echo _(echocardiogram)_ and it was inconclusive.

> **Doctor**: What was his interarm BP? _(blood pressure)_

> **Nurse**: Systolic was 140 on the right; 110 on the left.

> **Doctor**: Dammit, it's an aortic dissection! Get to the OR _(operating room)_ now!

> _(intense music playing)_

In this fictitious but [Shonda Rhimes-esque](https://en.wikipedia.org/wiki/Shonda_Rhimes#Grey's_Anatomy,_Private_Practice,_Scandal_and_other_projects_with_ABC) scenario, you might imagine the doctor going through a series of steps like a [flowchart](https://en.wikipedia.org/wiki/Flowchart), or a series of if-this-then-that steps to diagnose a patient. The first steps made the doctor ask what the interarm blood pressure was. Because interarm blood pressure took on the values it took on, the doctor diagnosed the patient with an aortic dissection.

Your goal, as a research biostatistical data scientist at the nation's top hospital, is to develop a medical test that can improve upon our current diagnosis system for [chronic kidney disease (CKD)](https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521).

**Real-world problem**: Develop a medical diagnosis test that is better than our current diagnosis system for CKD.

**Data science problem**: Develop a medical diagnosis test that reduces both the number of false positives and the number of false negatives.

---

## Step 2: Obtain the data.

### 1. Read in the data.

In [27]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/lab-4.02-classification-evaluation-master/chronic_kidney_disease_full.csv')

### 2. Check out the data dictionary. What are a few features or relationships you might be interested in checking out?

### Answer: Blood pressure, albumin, sugar, serum creatinine, potassium, hypertension, diabetes mellitus, coronary artery disease, anaemia



---

## Step 3: Explore the data.

### 3. How much of the data is missing from each column?

In [28]:
df.head()

age    bp     sg   al   su     rbc        pc         pcc          ba  \
0  48.0  80.0  1.020  1.0  0.0     NaN    normal  notpresent  notpresent   
1   7.0  50.0  1.020  4.0  0.0     NaN    normal  notpresent  notpresent   
2  62.0  80.0  1.010  2.0  3.0  normal    normal  notpresent  notpresent   
3  48.0  70.0  1.005  4.0  0.0  normal  abnormal     present  notpresent   
4  51.0  80.0  1.010  2.0  0.0  normal    normal  notpresent  notpresent   

     bgr  ...   pcv    wbcc  rbcc  htn   dm  cad  appet   pe  ane class  
0  121.0  ...  44.0  7800.0   5.2  yes  yes   no   good   no   no   ckd  
1    NaN  ...  38.0  6000.0   NaN   no   no   no   good   no   no   ckd  
2  423.0  ...  31.0  7500.0   NaN   no  yes   no   poor   no  yes   ckd  
3  117.0  ...  32.0  6700.0   3.9  yes   no   no   poor  yes  yes   ckd  
4  106.0  ...  35.0  7300.0   4.6   no   no   no   good   no   no   ckd  

[5 rows x 25 columns]

In [29]:
df.isnull().sum()

age        9
bp        12
sg        47
al        46
su        49
rbc      152
pc        65
pcc        4
ba         4
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wbcc     106
rbcc     131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
class      0
dtype: int64

### 4. Suppose that I dropped every row that contained at least one missing value. (In the context of analysis with missing data, we call this a "complete case analysis," because we keep only the complete cases!) How many rows would remain in our dataframe? What are at least two downsides to doing this?

> There's a good visual on slide 15 of [this deck](https://liberalarts.utexas.edu/prc/_files/cs/Missing-Data.pdf) that shows what a complete case analysis looks like if you're interested.

In [30]:
df.shape

(400, 25)

In [31]:
df_2 = df.dropna()
df_2.shape

(158, 25)

Answer: 158 rows. 

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     391 non-null    float64
 1   bp      388 non-null    float64
 2   sg      353 non-null    float64
 3   al      354 non-null    float64
 4   su      351 non-null    float64
 5   rbc     248 non-null    object 
 6   pc      335 non-null    object 
 7   pcc     396 non-null    object 
 8   ba      396 non-null    object 
 9   bgr     356 non-null    float64
 10  bu      381 non-null    float64
 11  sc      383 non-null    float64
 12  sod     313 non-null    float64
 13  pot     312 non-null    float64
 14  hemo    348 non-null    float64
 15  pcv     329 non-null    float64
 16  wbcc    294 non-null    float64
 17  rbcc    269 non-null    float64
 18  htn     398 non-null    object 
 19  dm      398 non-null    object 
 20  cad     398 non-null    object 
 21  appet   399 non-null    object 
 22  pe

### Dealing with missing values. I dropped a few columns that I did not feel were strongly associated with CKD and had a lot of missing values.

In [33]:
df.drop(['pc','rbc','pcv','sod','wbcc','rbcc'], axis=1, inplace=True)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     391 non-null    float64
 1   bp      388 non-null    float64
 2   sg      353 non-null    float64
 3   al      354 non-null    float64
 4   su      351 non-null    float64
 5   pcc     396 non-null    object 
 6   ba      396 non-null    object 
 7   bgr     356 non-null    float64
 8   bu      381 non-null    float64
 9   sc      383 non-null    float64
 10  pot     312 non-null    float64
 11  hemo    348 non-null    float64
 12  htn     398 non-null    object 
 13  dm      398 non-null    object 
 14  cad     398 non-null    object 
 15  appet   399 non-null    object 
 16  pe      399 non-null    object 
 17  ane     399 non-null    object 
 18  class   400 non-null    object 
dtypes: float64(10), object(9)
memory usage: 59.5+ KB


In [35]:
df_3 = df
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     391 non-null    float64
 1   bp      388 non-null    float64
 2   sg      353 non-null    float64
 3   al      354 non-null    float64
 4   su      351 non-null    float64
 5   pcc     396 non-null    object 
 6   ba      396 non-null    object 
 7   bgr     356 non-null    float64
 8   bu      381 non-null    float64
 9   sc      383 non-null    float64
 10  pot     312 non-null    float64
 11  hemo    348 non-null    float64
 12  htn     398 non-null    object 
 13  dm      398 non-null    object 
 14  cad     398 non-null    object 
 15  appet   399 non-null    object 
 16  pe      399 non-null    object 
 17  ane     399 non-null    object 
 18  class   400 non-null    object 
dtypes: float64(10), object(9)
memory usage: 59.5+ KB


In [36]:
df_3.fillna(0, inplace=True)
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     400 non-null    float64
 1   bp      400 non-null    float64
 2   sg      400 non-null    float64
 3   al      400 non-null    float64
 4   su      400 non-null    float64
 5   pcc     400 non-null    object 
 6   ba      400 non-null    object 
 7   bgr     400 non-null    float64
 8   bu      400 non-null    float64
 9   sc      400 non-null    float64
 10  pot     400 non-null    float64
 11  hemo    400 non-null    float64
 12  htn     400 non-null    object 
 13  dm      400 non-null    object 
 14  cad     400 non-null    object 
 15  appet   400 non-null    object 
 16  pe      400 non-null    object 
 17  ane     400 non-null    object 
 18  class   400 non-null    object 
dtypes: float64(10), object(9)
memory usage: 59.5+ KB


In [38]:
# drop rows where columns have '0' values for the following
df_3 = df[(df_3[['ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane']] != 0).all(axis=1)]

# print the resulting dataframe
print(df_3)


      age    bp     sg   al   su         pcc          ba    bgr    bu   sc  \
0    48.0  80.0  1.020  1.0  0.0  notpresent  notpresent  121.0  36.0  1.2   
1     7.0  50.0  1.020  4.0  0.0  notpresent  notpresent    0.0  18.0  0.8   
2    62.0  80.0  1.010  2.0  3.0  notpresent  notpresent  423.0  53.0  1.8   
3    48.0  70.0  1.005  4.0  0.0     present  notpresent  117.0  56.0  3.8   
4    51.0  80.0  1.010  2.0  0.0  notpresent  notpresent  106.0  26.0  1.4   
..    ...   ...    ...  ...  ...         ...         ...    ...   ...  ...   
395  55.0  80.0  1.020  0.0  0.0  notpresent  notpresent  140.0  49.0  0.5   
396  42.0  70.0  1.025  0.0  0.0  notpresent  notpresent   75.0  31.0  1.2   
397  12.0  80.0  1.020  0.0  0.0  notpresent  notpresent  100.0  26.0  0.6   
398  17.0  60.0  1.025  0.0  0.0  notpresent  notpresent  114.0  50.0  1.0   
399  58.0  80.0  1.025  0.0  0.0  notpresent  notpresent  131.0  18.0  1.1   

     pot  hemo  htn   dm cad appet   pe  ane   class  
0    0.0

In [39]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 393 entries, 0 to 399
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     393 non-null    float64
 1   bp      393 non-null    float64
 2   sg      393 non-null    float64
 3   al      393 non-null    float64
 4   su      393 non-null    float64
 5   pcc     393 non-null    object 
 6   ba      393 non-null    object 
 7   bgr     393 non-null    float64
 8   bu      393 non-null    float64
 9   sc      393 non-null    float64
 10  pot     393 non-null    float64
 11  hemo    393 non-null    float64
 12  htn     393 non-null    object 
 13  dm      393 non-null    object 
 14  cad     393 non-null    object 
 15  appet   393 non-null    object 
 16  pe      393 non-null    object 
 17  ane     393 non-null    object 
 18  class   393 non-null    object 
dtypes: float64(10), object(9)
memory usage: 61.4+ KB


In [40]:

# show unique values for each column
print(df_3['ba'].unique())
print(df_3['htn'].unique())
print(df_3['dm'].unique())
print(df_3['cad'].unique())
print(df_3['appet'].unique())
print(df_3['pe'].unique())
print(df_3['ane'].unique())


['notpresent' 'present']
['yes' 'no']
['yes' 'no']
['no' 'yes']
['good' 'poor']
['no' 'yes']
['no' 'yes']


In [41]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 393 entries, 0 to 399
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     393 non-null    float64
 1   bp      393 non-null    float64
 2   sg      393 non-null    float64
 3   al      393 non-null    float64
 4   su      393 non-null    float64
 5   pcc     393 non-null    object 
 6   ba      393 non-null    object 
 7   bgr     393 non-null    float64
 8   bu      393 non-null    float64
 9   sc      393 non-null    float64
 10  pot     393 non-null    float64
 11  hemo    393 non-null    float64
 12  htn     393 non-null    object 
 13  dm      393 non-null    object 
 14  cad     393 non-null    object 
 15  appet   393 non-null    object 
 16  pe      393 non-null    object 
 17  ane     393 non-null    object 
 18  class   393 non-null    object 
dtypes: float64(10), object(9)
memory usage: 61.4+ KB


### 5. Thinking critically about how our data were gathered, it's likely that these records were gathered by doctors and nurses. Brainstorm three potential areas (in addition to the missing data we've already discussed) where this data might be inaccurate or imprecise.

Answer: 
1. Human error on the part of health care workers
2. Different practices of recording data in different hospitals/clinics. No universal standards, so there may be error in merging the data. 
3. Health records were not properly QC'd and missing values went unnoticed.
4. Maybe the test panels ordered were not all the same. 
5. Test subject refusal to respond to age question. 

---

## Step 4: Model the data.

### 6. Suppose that I want to construct a model where no person who has CKD will ever be told that they do not have CKD. What (very simple, no machine learning needed) model can I create that will never tell a person with CKD that they do not have CKD?

> Hint: Don't think about `statsmodels` or `scikit-learn` here.

Answer: A model that predicts that person always has CKD no matter the health condition. This will create false positives. This is the least harmful model becuase it minimizes false negatives. 

### 7. In problem 6, what common classification metric did we optimize for? Did we minimize false positives or negatives?

Answer: We optimized for sensitvity. False negatives were minimized. 

### 8. Thinking ethically, what is at least one disadvantage to the model you described in problem 6?

Answer:A high number of false positives, wasting money and treating people who don't need treatment. The usual treatment for CKD is some type of blood pressure medication. Those come with their own set of risks. 

### 9. Suppose that I want to construct a model where a person who does not have CKD will ever be told that they do have CKD. What (very simple, no machine learning needed) model can I create that will accomplish this?

Answer: Create a model where a person does not have CKD unless the Glomular Filtration Rate is less than 60 for 3 or more months. This creates a situation where people have CKD and they are unaware. 

### 10. In problem 9, what common classification metric did we optimize for? Did we minimize false positives or negatives?

Answer: We minimized for false positives but this hurts certain populations. 

### 11. Thinking ethically, what is at least one disadvantage to the model you described in problem 9?

Answer: This is problematic because a person could have a GFR rate of less than 60 with one doctor. Perhaps they change doctors and go for a visit a year later with the new docotor to have a GFR rate of less than 60. Well, most people don't get a panel blood test done every month so a person could have a GFR of less than 60 for years and it will not be caught until more serious issues arise like hypertension or worse signs of kidney failure. Also GFR rates for people of African descent are thought to be under-estimated. Interpreting GFR rates for people of African descent can be tricky so waiting it out may be harmful. This would cause a lot of false negatives. 

### 12. Construct a logistic regression model in `sklearn` predicting class from the other variables. You may scale, select/drop, and engineer features as you wish - build a good model! Make sure, however, that you include at least one categorical/dummy feature and at least one quantitative feature.

> Hint: Remember to do a train/test split!

In [42]:
cols_to_dummy = ['htn', 'dm', 'cad', 'appet', 'pe', 'ane','ba', 'pcc']

for col in cols_to_dummy:
    dummies = pd.get_dummies(df_3[col], prefix=col, drop_first=True)
    df_3 = pd.concat([df_3, dummies], axis=1)

df_3.drop(cols_to_dummy, axis=1, inplace=True)


In [43]:
#change class to 0 and 1. NoCkd = 1, Ckd =0
df_3['class'].unique()

array(['ckd', 'notckd'], dtype=object)

In [44]:
df_3['class'] = df_3['class'].replace({'ckd': 1, 'notckd': 0})
df_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 393 entries, 0 to 399
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          393 non-null    float64
 1   bp           393 non-null    float64
 2   sg           393 non-null    float64
 3   al           393 non-null    float64
 4   su           393 non-null    float64
 5   bgr          393 non-null    float64
 6   bu           393 non-null    float64
 7   sc           393 non-null    float64
 8   pot          393 non-null    float64
 9   hemo         393 non-null    float64
 10  class        393 non-null    int64  
 11  htn_yes      393 non-null    uint8  
 12  dm_yes       393 non-null    uint8  
 13  cad_yes      393 non-null    uint8  
 14  appet_poor   393 non-null    uint8  
 15  pe_yes       393 non-null    uint8  
 16  ane_yes      393 non-null    uint8  
 17  ba_present   393 non-null    uint8  
 18  pcc_present  393 non-null    uint8  
dtypes: float

In [64]:
# Features and target variable
y = df_3['class']
X = df_3.iloc[:, 11:19]





In [65]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Define features and Target variables
X = df_3.iloc[:, 11:19]
y = df_3['class']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate logistic regression model and fit to training data
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Predict on testing data and calculate accuracy score
y_pred = logreg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy score:", accuracy)


Accuracy score: 0.9240506329113924


In [47]:
logreg.score(X_train, y_train)

0.910828025477707

In [48]:
logreg.score(X_test, y_test)

0.9240506329113924

---

## Step 5: Evaluate the model.

### 13. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your quantitative features.

In [49]:
logreg.coef_

array([[2.55780507, 2.70886441, 0.68782444, 1.81361636, 2.05613389,
        1.61478859, 0.66858586, 1.46730047]])

### 14. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your categorical/dummy features.

In [80]:
#htn - Hypertension
import numpy as np
np.exp(2.557)

12.89706801660682

Answer: A person presenting with hypertension would have an almost 13 times greater likelihood or probability of having CKD compared to those without hypertension. Assuming everything else is the same.  

### 15. Despite being a relatively simple model, logistic regression is very widely used in the real world. Why do you think that's the case? Name at least two advantages to using logistic regression as a modeling technique.

## Answer:The simplicity makes Logistic regressions easy to interpret because it is binary or based on probability. The impact of each variable can be easily understood. Large data sets can be trained in a short ammount of time. **Source:**  file:///C:/Users/bridg/AppData/Local/Temp/Temp1_lab-4.02-classification-evaluation-master.zip/lab-4.02-classification-evaluation-master/chronic_kidney_disease.pdf

### 16. Does it make sense to generate a confusion matrix on our training data or our test data? Why? Generate it on the proper data.

> Hint: Once you've generated your predicted $y$ values and you have your observed $y$ values, then it will be easy to [generate a confusion matrix using sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

In [51]:
from sklearn.metrics import confusion_matrix

y_pred = logreg.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

print(cm)


[[22  0]
 [ 6 51]]


In [54]:

# create confusion matrix df
cm = pd.DataFrame(cm, columns=['Pred Negative','Pred Positive'], index=['Actual Negative','Actual Positive'])
cm

Pred Negative  Pred Positive
Actual Negative             22              0
Actual Positive              6             51

### 17. In this hospital case, we want to predict CKD. Do we want to optimize for sensitivity, specificity, or something else? Why? (If you don't think there's one clear answer, that's okay! There rarely is. Be sure to defend your conclusion!)

Answer:Optimize for specificity to avoid false positives for the general population. For sub-groups of the population where there prevalence is much higher, perhaps a different approach should be taken. For the general population, this would make it less likely to label someone with CKD who may not have it. 

### 18 (BONUS). Write a function that will create an ROC curve for you, then plot the ROC curve.

Here's a strategy you might consider:
1. In order to even begin, you'll need some fit model. Use your logistic regression model from problem 12.
2. We want to look at all values of your "threshold" - that is, anything where .predict() gives you above your threshold falls in the "positive class," and anything that is below your threshold falls in the "negative class." Start the threshold at 0.
3. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
4. Increment your threshold by some "step." Maybe set your step to be 0.01, or even smaller.
5. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
6. Repeat steps 3 and 4 until you get to the threshold of 1.
7. Plot the values of sensitivity and 1 - specificity.

### 19. Suppose you're speaking with the biostatistics lead at Mayo Clinic, who asks you "Why are unbalanced classes generally a problem? Are they a problem in this particular CKD analysis?" How would you respond?

Answer: Unbalanced classes would perform poorly on minority classes. The unbalanced classes in this study are not truly a problem because the number of samples without CKD is 37% which isn't too bad. 

In [57]:
df_3['class'].value_counts(normalize=True) * 100


1    63.613232
0    36.386768
Name: class, dtype: float64

### 20. Suppose you're speaking with a doctor at Mayo Clinic who, despite being very smart, doesn't know much about data science or statistics. How would you explain why unbalanced classes are generally a problem to this doctor?

Answer:Unbalanced classes are when one group significantly outnumbers the other in a study. In this study, there were more samples with CKD than without CKD. A model trained with unbalanced classes wouldn't do a good job of predicting for the smaller group and may not work well on new data or in real life.  

### 21. Let's create very unbalanced classes just for the sake of this example! Generate very unbalanced classes by [bootstrapping](http://stattrek.com/statistics/dictionary.aspx?definition=sampling_with_replacement) (a.k.a. random sampling with replacement) the majority class.

1. The majority class are those individuals with CKD.
2. Generate a random sample of size 200,000 of individuals who have CKD **with replacement**. (Consider setting a random seed for this part!)
3. Create a new dataframe with the original data plus this random sample of data.
4. Now we should have a dataset with around 200,000 observations, of which only about 0.00075% are non-CKD individuals.

In [59]:
df_3['class'].value_counts()

1    250
0    143
Name: class, dtype: int64

In [60]:
import numpy as np

np.random.seed(42)  # set random seed 

# Get minority class data
minority_class = df_3[df_3['class'] == 0]

# Get majority class data
majority_class = df_3[df_3['class'] == 1]

# Bootstrap majority class
n_samples = 200000 - 143
bootstrap_sample = majority_class.sample(n_samples, replace=True, random_state=42)

# Concat the minority and majority class 
bs_df = pd.concat([minority_class, bootstrap_sample])

# Shuffle data
bs_df = bs_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Check class distribution
print(bs_df['class'].value_counts(normalize=True))


1    0.999285
0    0.000715
Name: class, dtype: float64


In [61]:
bs_df.head()

age    bp     sg   al   su    bgr     bu    sc  pot  hemo  class  htn_yes  \
0  72.0  90.0  1.010  2.0  0.0  124.0   53.0   2.3  0.0  11.9      1        0   
1  65.0  90.0  1.010  4.0  2.0  172.0   82.0  13.5  6.3   8.8      1        1   
2  48.0  80.0  1.005  4.0  0.0  133.0  139.0   8.5  5.5  10.3      1        0   
3  50.0  70.0  1.020  0.0  0.0  109.0   32.0   1.4  4.7   0.0      1        0   
4  34.0  60.0  1.020  0.0  0.0  117.0   28.0   2.2  3.8   0.0      1        0   

   dm_yes  cad_yes  appet_poor  pe_yes  ane_yes  ba_present  pcc_present  
0       0        0           0       0        0           0            1  
1       1        0           0       1        1           0            0  
2       1        0           0       1        0           1            0  
3       0        0           1       0        0           0            0  
4       0        0           0       1        0           0            0

### 22. Build a logistic regression model on the unbalanced class data and evaluate its performance using whatever method(s) you see fit. How would you describe the impact of unbalanced classes on logistic regression as a classifier?
> Be sure to look at how well it performs on non-CKD data.

In [68]:
#define features (Same as before, just using bootstrap dataframe)
y_bs = bs_df['class']
X_bs = bs_df.iloc[:, 11:19]

In [69]:
print(X_bs)

        htn_yes  dm_yes  cad_yes  appet_poor  pe_yes  ane_yes  ba_present  \
0             0       0        0           0       0        0           0   
1             1       1        0           0       1        1           0   
2             0       1        0           0       1        0           1   
3             0       0        0           1       0        0           0   
4             0       0        0           0       1        0           0   
...         ...     ...      ...         ...     ...      ...         ...   
199995        1       1        0           0       1        0           0   
199996        1       1        0           1       1        0           0   
199997        1       1        0           0       1        0           0   
199998        1       1        0           1       1        0           0   
199999        1       1        0           1       1        1           0   

        pcc_present  
0                 1  
1                 0  
2        

In [70]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_bs, y_bs, test_size=0.2, random_state=42)

# Instantiate logistic regression model and fit to training data
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Predict and calculate accuracy score
y_pred = logreg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy score:", accuracy)

Accuracy score: 0.9993


In [72]:
logreg.score(X_train, y_train)

0.99928125

In [73]:
logreg.score(X_test, y_test)

0.9993

In [75]:
logreg.coef_

array([[2.76382538, 2.94715989, 0.65039512, 2.1594704 , 2.45522898,
        1.89352913, 0.90968529, 2.15477211]])

In [76]:
y_pred = logreg.predict(X_test)
cm_bs = confusion_matrix(y_test, y_pred)

print(cm_bs)


[[    0    28]
 [    0 39972]]


The model is overfitting to the CKD class. The model wouldn't handle new data well. It failed to identify non-CKD cases. 

---

## Step 6: Answer the problem.

At this step, you would generally answer the problem! In this situation, you would likely present your model to doctors or administrators at the hospital and show how your model results in reduced false positives/false negatives. Next steps would be to find a way to roll this model and its conclusions out across the hospital so that the outcomes of patients with CKD (and without CKD!) can be improved!

Ran out of time to answer this question. I still need to do the other homeowrk and only have 2 hours.